In [ ]:
!pip  install transformers


     |████████████████████████████████| 3.5 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 25.5 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
     |████████████████████████████████| 67 kB 3.2 MB/s 
     |████████████████████████████████| 6.8 MB 21.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW

In [ ]:
import os
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class Xlm_roberta_Model(nn.Module):
    def __init__(self, model_name):
        super(Xlm_roberta_Model, self).__init__()
        self.model = AutoModel.from_pretrained(model_name, output_hidden_states=True)
        output_channel = 16  # number of kernels
        num_classes = 1  # number of targets to predict
        dropout = 0.2  # dropout value
        embedding_dim = 768   # length of embedding dim

        ks = 3  # three conv nets here

        # input_channel = word embeddings at a value of 1; 3 for RGB images
        input_channel = 4  # for single embedding, input_channel = 1

        # [3, 4, 5] = window height
        # padding = padding to account for height of search window

        # 3 convolutional nets
        self.conv1 = nn.Conv2d(input_channel, output_channel, (3, embedding_dim), padding=(2, 0), groups=4)
        self.conv2 = nn.Conv2d(input_channel, output_channel, (4, embedding_dim), padding=(3, 0), groups=4)
        self.conv3 = nn.Conv2d(input_channel, output_channel, (5, embedding_dim), padding=(4, 0), groups=4)

        # apply dropout
        self.dropout = nn.Dropout(dropout)

        # fully connected layer for classification
        # 3x conv nets * output channel
        self.fc1 = nn.Linear(ks * output_channel, num_classes)

    def forward(self, text_id, text_mask):
        # get the last 4 layers
        outputs= self.model(text_id, attention_mask=text_mask)
        # all_layers  = [4, 16, 256, 768]
        # print(outputs)
        hidden_layers = outputs[-1]  # get hidden layers

        hidden_layers = torch.stack(hidden_layers, dim=1)
        x = hidden_layers[:, -7:-3] 
        # x = x.unsqueeze(1)
        # x = torch.mean(x, 0)
        # print(hidden_layers.size())
      
        torch.cuda.empty_cache()
        x = [F.relu(self.conv1(x)).squeeze(3), F.relu(self.conv2(x)).squeeze(3), F.relu(self.conv3(x)).squeeze(3)]
        x = [F.dropout(i,0.65) for i in x]
        # max-over-time pooling; # (batch, channel_output) * ks
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        # concat results; (batch, channel_output * ks)
        x = torch.cat(x, 1)
        # add dropout
        x = self.dropout(x)
        # generate logits (batch, target_size)
        logit = self.fc1(x)
        torch.cuda.empty_cache()
        return logit
        

In [ ]:
class RobertaModel(nn.Module):
    def __init__(self, model_name):
        super(RobertaModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, 1)
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        return outputs

In [ ]:
class JigsawDatasetTest(Dataset):
    def __init__(self, df, tokenizer, max_length,column_):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df[column_].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
       
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
       
        
        
        return {
            'text_ids': torch.tensor(ids, dtype=torch.long),
            'text_mask': torch.tensor(mask, dtype=torch.long),
           
        }

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['text_ids'].to(device, dtype = torch.long)
        mask = data['text_mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        sig=nn.Sigmoid()
        outputs=sig(outputs)
        # outputs = outputs.argmax(dim=1)
#         print(len(outputs))
#         print(len(np.max(outputs.cpu().detach().numpy(),axis=1)))
        PREDS.append(outputs.detach().cpu().numpy()) 
        # print(outputs.detach().cpu().numpy())
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference_xlm_roberta(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = Xlm_roberta_Model('xlm-roberta-base')
        model.to('cuda')
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    # print(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    # print(final_preds)
    final_preds[final_preds>=0.5] = 1
    final_preds[final_preds<0.5] = 0
    # final_preds= np.argmax(final_preds,axis=1)
    return final_preds

In [ ]:
def inference_Roberta(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = RobertaModel('roberta-base')
        model.to('cuda')
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    # print(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    # print(final_preds)
    final_preds[final_preds>=0.5] = 1
    final_preds[final_preds<0.5] = 0
    # final_preds= np.argmax(final_preds,axis=1)
    return final_preds

In [ ]:
arabert_tokenizer=AutoTokenizer.from_pretrained('xlm-roberta-base')

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

In [ ]:
marbert_tokenizer=AutoTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
class RobertaModel_kim(nn.Module):
    def __init__(self, model_name):
        super(RobertaModel_kim, self).__init__()
        self.model = AutoModel.from_pretrained(model_name, output_hidden_states=True)
        output_channel = 16  # number of kernels
        num_classes = 1  # number of targets to predict
        dropout = 0.2  # dropout value
        embedding_dim = 768   # length of embedding dim

        ks = 3  # three conv nets here

        # input_channel = word embeddings at a value of 1; 3 for RGB images
        input_channel = 4  # for single embedding, input_channel = 1

        # [3, 4, 5] = window height
        # padding = padding to account for height of search window

        # 3 convolutional nets
        self.conv1 = nn.Conv2d(input_channel, output_channel, (3, embedding_dim), padding=(2, 0), groups=4)
        self.conv2 = nn.Conv2d(input_channel, output_channel, (4, embedding_dim), padding=(3, 0), groups=4)
        self.conv3 = nn.Conv2d(input_channel, output_channel, (5, embedding_dim), padding=(4, 0), groups=4)

        # apply dropout
        self.dropout = nn.Dropout(dropout)

        # fully connected layer for classification
        # 3x conv nets * output channel
        self.fc1 = nn.Linear(ks * output_channel, num_classes)

    def forward(self, text_id, text_mask):
        # get the last 4 layers
        outputs= self.model(text_id, attention_mask=text_mask)
        # all_layers  = [4, 16, 256, 768]
        # print(outputs)
        hidden_layers = outputs[-1]  # get hidden layers

        hidden_layers = torch.stack(hidden_layers, dim=1)
        x = hidden_layers[:, -7:-3] 
        # x = x.unsqueeze(1)
        # x = torch.mean(x, 0)
        # print(hidden_layers.size())
      
        torch.cuda.empty_cache()
        x = [F.relu(self.conv1(x)).squeeze(3), F.relu(self.conv2(x)).squeeze(3), F.relu(self.conv3(x)).squeeze(3)]
        x = [F.dropout(i,0.65) for i in x]
        # max-over-time pooling; # (batch, channel_output) * ks
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        # concat results; (batch, channel_output * ks)
        x = torch.cat(x, 1)
        # add dropout
        x = self.dropout(x)
        # generate logits (batch, target_size)
        logit = self.fc1(x)
        torch.cuda.empty_cache()
        return logit

In [ ]:
def inference_Roberta_kim(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = RobertaModel_kim('roberta-base')
        model.to('cuda')
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    # print(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    # print(final_preds)
    final_preds[final_preds>=0.5] = 1
    final_preds[final_preds<0.5] = 0
    # final_preds= np.argmax(final_preds,axis=1)
    return final_preds

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/ISarcasm/Test_dataset/taskC.En.input.csv')

In [ ]:
test_df

,text_0,text_1
0,I see that your team played well today!,I'm sorry that your team didn't win yesterday.
1,"Anthony Taylor is such a fair referee, I wish ...",I hope Anthony Taylor is never put in charge o...
2,"the weather is gloomy, just raining and dull.",What a glorious weather today
3,People going out to get there boosters without...,Nice to see the sheep getting their boosters t...
4,"Really great weather we're having, love a bit ...",Really cold January so far - looking forward t...
...,...,...
195,"the tories betrayed the nation, what a surprise!","the tories betrayed the nation, as expected"
196,Cant believe we have to spend the rest of our ...,Cant wait to spend the rest of my life waiting...
197,Isn't it just amazing how competent the govern...,"Everything is a total mess, how can anyone be ..."
198,Thanks Boris Johnson for restricting travel ab...,The reasoning behind the tightening of travel ...


In [ ]:
test_df.columns

Index(['text_0', 'text_1'], dtype='object')

In [ ]:
test_dataset = JigsawDatasetTest(test, tokenizer=arabert_tokenizer, max_length=128,column_='text_0')
test_loader = DataLoader(test_dataset, batch_size=16, 
                              num_workers=2, shuffle=False, pin_memory=True)

In [ ]:

MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_arabert_first_column = inference_xlm_roberta(MODEL_PATH_2, test_loader, 'cuda')

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 13/13 [00:06<00:00,  2.15it/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


...

100%|██████████| 13/13 [00:05<00:00,  2.25it/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 13/13 [00:05<00:00,  2.17it/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 13/13 [00:06<00:00,  2.16it/s]


In [ ]:
test_dataset = JigsawDatasetTest(test, tokenizer=arabert_tokenizer, max_length=128,column_='text_1')
test_loader = DataLoader(test_dataset, batch_size=16, 
                              num_workers=2, shuffle=False, pin_memory=True)

In [ ]:

MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/xlm_roberta_kim/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_arabert_second_column = inference_xlm_roberta(MODEL_PATH_2, test_loader, 'cuda')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 13/13 [00:05<00:00,  2.24it/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


100%|██████████| 13/13 [00:05<00:00,  2.24it/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

100%|██████████| 13/13 [00:05<00:00,  2.25it/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 13/13 [00:05<00:00,  2.19it/s]
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 13/13 [00:05<00:00,  2.17it/s]


In [ ]:
df_arabert = pd.DataFrame(list(zip(preds_arabert_first_column.flatten(), preds_arabert_second_column.flatten())),columns =['Prediction_text_0','Prediction_text_1'])

In [ ]:
prediction_list=df_arabert.idxmax(axis='columns').to_list()

In [ ]:
prediction_final=[]
for pred in prediction_list:
  if pred=='Prediction_text_0':
    prediction_final.append(0)
  else:
    prediction_final.append(1)



In [ ]:
print(print_statistics(test['sarcastic_id'],prediction_final))

Accuracy: 0.730
Precision: 0.810
Recall: 0.730
F_score: 0.704

              precision    recall  f1-score   support

           0       0.67      0.99      0.80       107
           1       0.98      0.43      0.60        93

    accuracy                           0.73       200
   macro avg       0.82      0.71      0.70       200
weighted avg       0.81      0.73      0.70       200

(0.73, 0.8103252032520325, 0.73, 0.7040029177421164)


In [ ]:
test_dataset = JigsawDatasetTest(test, tokenizer=marbert_tokenizer, max_length=128,column_='text_0')
test_loader = DataLoader(test_dataset, batch_size=16, 
                              num_workers=2, shuffle=False, pin_memory=True)

In [ ]:

MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_marbert_first_column = inference_Roberta(MODEL_PATH_2, test_loader, 'cuda')

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 13/13 [00:03<00:00,  3.35it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


...

100%|██████████| 13/13 [00:03<00:00,  3.34it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 13/13 [00:03<00:00,  3.35it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 13/13 [00:03<00:00,  3.33it/s]


In [ ]:

MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_marbert_kim_first_column = inference_Roberta_kim(MODEL_PATH_2, test_loader, 'cuda')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 13/13 [00:05<00:00,  2.24it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


100%|██████████| 13/13 [00:05<00:00,  2.29it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

100%|██████████| 13/13 [00:05<00:00,  2.28it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 13/13 [00:05<00:00,  2.29it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 13/13 [00:05<00:00,  2.29it/s]


In [ ]:
test_dataset = JigsawDatasetTest(test, tokenizer=marbert_tokenizer, max_length=128,column_='text_1')
test_loader = DataLoader(test_dataset, batch_size=16, 
                              num_workers=2, shuffle=False, pin_memory=True)

In [ ]:

MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_marbert_second_column = inference_Roberta(MODEL_PATH_2, test_loader, 'cuda')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 13/13 [00:03<00:00,  3.37it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


100%|██████████| 13/13 [00:03<00:00,  3.38it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

100%|██████████| 13/13 [00:03<00:00,  3.39it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 13/13 [00:03<00:00,  3.38it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 13/13 [00:03<00:00,  3.36it/s]


In [ ]:

MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/TaskC_models/roberta_model_EN_kmicnn/Loss-Fold-4.bin']
# MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models_Task_B/bert_tweet_kim_cnn/Loss-Fold-0.bin']
preds_marbert_kim_second_column = inference_Roberta_kim(MODEL_PATH_2, test_loader, 'cuda')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 13/13 [00:05<00:00,  2.28it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


100%|██████████| 13/13 [00:05<00:00,  2.29it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

100%|██████████| 13/13 [00:05<00:00,  2.30it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 13/13 [00:05<00:00,  2.28it/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 13/13 [00:05<00:00,  2.28it/s]


In [ ]:
preds_marbert= pd.DataFrame(list(zip(preds_marbert_first_column.flatten(), preds_marbert_second_column.flatten())),columns =['Prediction_text_0','Prediction_text_1'])

In [ ]:
prediction_list=preds_marbert.idxmax(axis='columns').to_list()

In [ ]:
prediction_final=[]
for pred in prediction_list:
  if pred=='Prediction_text_0':
    prediction_final.append(0)
  else:
    prediction_final.append(1)



In [ ]:
print(print_statistics(test['sarcastic_id'],prediction_final))

Accuracy: 0.695
Precision: 0.782
Recall: 0.695
F_score: 0.660

              precision    recall  f1-score   support

           0       0.64      0.98      0.77       107
           1       0.94      0.37      0.53        93

    accuracy                           0.69       200
   macro avg       0.79      0.67      0.65       200
weighted avg       0.78      0.69      0.66       200

(0.695, 0.7816971544715446, 0.695, 0.6596919248262251)


In [ ]:
preds_marbert_kim = pd.DataFrame(list(zip(preds_marbert_kim_first_column.flatten(), preds_marbert_kim_second_column.flatten())),columns =['Prediction_text_0','Prediction_text_1'])

In [ ]:
prediction_list=preds_marbert_kim.idxmax(axis='columns').to_list()

In [ ]:
prediction_final=[]
for pred in prediction_list:
  if pred=='Prediction_text_0':
    prediction_final.append(0)
  else:
    prediction_final.append(1)



In [ ]:
print(print_statistics(test['sarcastic_id'],prediction_final))

Accuracy: 0.720
Precision: 0.796
Recall: 0.720
F_score: 0.693

              precision    recall  f1-score   support

           0       0.66      0.98      0.79       107
           1       0.95      0.42      0.58        93

    accuracy                           0.72       200
   macro avg       0.81      0.70      0.69       200
weighted avg       0.80      0.72      0.69       200

(0.72, 0.7956189599631844, 0.72, 0.6930400628436764)
